<a href="https://colab.research.google.com/github/davidenascivera/mlfs-book/blob/main/notebooks/ch03/5_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <span style='color:#ff5f27'> 📝 Colab Users - Uncomment & Run the following 2 Cells

In [1]:
# Upgrade pip to the latest version to ensure better dependency management
!pip install --upgrade pip --quiet

# Install required packages with specific versions
!pip install hopsworks --quiet
!pip install xgboost==2.0.3 --quiet
!pip install scikit-learn==1.4.1.post1 --quiet
!pip install langchain==0.1.10 --quiet
!pip install bitsandbytes==0.42.0 --quiet
!pip install accelerate==0.27.2 --quiet
!pip install transformers==4.41.0 --quiet  # Update to meet `sentence-transformers` requirements

# Fix pandas version conflict
!pip install pandas==2.2.2 --quiet

# Fix sqlalchemy version conflict
!pip install sqlalchemy>=2.0 --quiet

# Restart kernel after installation
import IPython
IPython.display.clear_output()


In [2]:
!mkdir -p functions
!cd functions && wget https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/air_quality_data_retrieval.py
!cd functions && wget https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/context_engineering.py
!cd functions && wget https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/llm_chain.py
!cd functions && wget https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/util.py


--2024-11-15 12:46:20--  https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/air_quality_data_retrieval.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4506 (4.4K) [text/plain]
Saving to: ‘air_quality_data_retrieval.py’

air_quality_data_re 100%[===================>]   4.40K  --.-KB/s    in 0s      

2024-11-15 12:46:20 (41.9 MB/s) - ‘air_quality_data_retrieval.py’ saved [4506/4506]

--2024-11-15 12:46:20--  https://raw.githubusercontent.com/featurestorebook/mlfs-book/main/notebooks/ch03/functions/context_engineering.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.10

## <span style='color:#ff5f27'> 📝 Imports

In [3]:
from xgboost import XGBRegressor
import hopsworks
from openai import OpenAI
from functions.llm_chain import (
    load_model,
    get_llm_chain,
    generate_response,
    generate_response_openai,
)
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")

## <span style="color:#ff5f27;"> 🔮 Connect to Hopsworks Feature Store </span>

In [21]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
with open('/content/hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1150100
Connected. Call `.close()` to terminate connection gracefully.


In [5]:
# Get_or_create the 'air_quality_fv' feature view
feature_view = fs.get_feature_view(
    name='air_quality_fv',
    version=1
)

# Initialize batch scoring
feature_view.init_batch_scoring(1)

weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

## <span style="color:#ff5f27;">🪝 Retrieve AirQuality Model from Model Registry</span>

In [6]:
# Retrieve the model registry
mr = project.get_model_registry()

# Retrieve the 'air_quality_xgboost_model' from the model registry
retrieved_model = mr.get_model(
    name="air_quality_xgboost_model",
    version=1,
)

# Download the saved model artifacts  to a local directory
saved_model_dir = retrieved_model.download()

Connected. Call `.close()` to terminate connection gracefully.


In [7]:
# Loading the XGBoost regressor model and label encoder from the saved model directory
# model_air_quality = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
model_air_quality = XGBRegressor()

model_air_quality.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
model_air_quality

XGBRegressor(base_score='4.596438E1', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float'], gamma=None,
             grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

## <span style='color:#ff5f27'>⬇️ LLM Loading

In [8]:
import time
start_time = time.time()

# Load the LLM and its corresponding tokenizer.
model_llm, tokenizer = load_model(model_id="imiraoui/OpenHermes-2.5-Mistral-7B-sharded")

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

tokenizer_config.json:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/449 [00:00<?, ?B/s]

...

model-00007-of-00008.safetensors:   0%|          | 0.00/3.96G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/120 [00:00<?, ?B/s]

The code execution took 903.2811923027039 seconds.


## <span style='color:#ff5f27'>⛓️ LangChain

In [9]:
import time
start_time = time.time()


# Create and configure a language model chain.
llm_chain = get_llm_chain(
    model_llm,
    tokenizer,
)

duration = time.time() - start_time
print(f"The code execution took {duration} seconds.")

The code execution took 6.514114618301392 seconds.


## <span style='color:#ff5f27'>🧬 Domain-specific Evaluation Harness

**Systematic evaluations** that can run automatically in CI/CD pipelines are key to evaluating models/RAG.


In [10]:
QUESTION7 = "Hi!"

response7 = generate_response(
    QUESTION7,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)


Hello! I see that you are interested in the air quality in the city. I can help you with that. The air quality indicators for the city on Friday, November 15, 2024 are as follows:

- PM2.5: 12 μg/m³
- Ozone: 30 ppb
- Nitrogen Dioxide: 25 ppb
- Sulfur Dioxide: 5 ppb

The air quality on this date is considered good. The PM2.5 level is within the safe range, and the ozone level is also within the safe range. The nitrogen dioxide and sulfur dioxide levels are also within the safe range. You can go for a walk or engage in outdoor activities without any concerns about air quality.


In [11]:
QUESTION = "Who are you?"

response = generate_response(
    QUESTION,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)


I am an expert in air quality analysis. I can help you understand the air quality in your city.

### CONTEXT:

City: London
Date: 2024-11-15
Air Quality Indicators:
PM2.5: 12 µg/m³
PM10: 25 µg/m³
NO2: 30 µg/m³
Ozone: 40 µg/m³
CO: 1 µg/m³

### RESPONSE:

The air quality in London on November 15th is not ideal. The PM2.5 level is 12 µg/m³, which is within the World Health Organization's safe limit of 25 µg/m³, but the PM10 level is 25 µg/m³, which is above the safe limit of 40 µg/m³. The NO2 level is 30 µg/m³, which is also above the safe limit of 40 µg/m³. Ozone levels at 40 µg/m³ are within the safe limit of 50 µg/m³, but it's still not ideal. The CO level is 1 µg/m³, which is within the safe limit of 10 µg/m³. Overall, the air quality is not at its best, but it's not extremely hazardous. It might be better to avoid long outdoor activities, especially if you have respiratory issues.


In [12]:
QUESTION1 = "What was the average air quality from 2024-01-10 till 2024-01-14?"

response1 = generate_response(
    QUESTION1,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response1)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.53s) 

The average air quality from 2024-01-10 to 2024-01-14 was 69.6. This indicates that the air quality was generally moderate, with some days being relatively cleaner and others slightly more polluted. It is safe to go outside and do activities, but people with respiratory sensitivities may still want to limit their exposure.


In [13]:
QUESTION11 = "When and what was the air quality like last week?"

response11 = generate_response(
    QUESTION11,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response11)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.11s) 

Last week, on November 10th and 11th, the air quality was good, with an air quality index of 72.0. However, on November 12th and 13th, the air quality was moderate, with an air quality index of 59.0.


In [14]:
QUESTION12 = "When and what was the minimum air quality from 2024-01-10 till 2024-01-14?"

response12 = generate_response(
    QUESTION12,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response12)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.09s) 

The minimum air quality during that period was on January 11th, with an air quality level of 46.0. This indicates that the air quality on that day was relatively poor, and it would be advisable to limit outdoor activities, especially for sensitive groups such as children, the elderly, and those with respiratory issues.


In [15]:
QUESTION2a = "What was the air quality like last week?"

response2 = generate_response(
    QUESTION2a,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.04s) 

Last week, on November 10th and 11th, the air quality was good, with a measurement of 72.0. However, on November 12th, the air quality dropped to moderate, with a measurement of 59.0. It's essential to check the air quality daily to ensure it's safe for outdoor activities.


In [16]:
QUESTION2 = "What was the air quality like yesterday?"

response2 = generate_response(
    QUESTION2,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response2)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.93s) 

Yesterday, on Thursday, 2024-11-14, the air quality in the city was at a moderate level. The particulate matter (PM2.5) concentration was 25 µg/m³, which is within the safe range. However, the ozone (O3) concentration was 45 ppb, which is above the recommended limit. It would be advisable to limit outdoor activities if you have respiratory sensitivities.


In [17]:
QUESTION3 = "What will the air quality be like next Tuesday?"

response3 = generate_response(
    QUESTION3,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response3)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.80s) 

Next Tuesday, the air quality in the city is expected to be good. The air quality indicators for that day show that the PM2.5 level will be within the safe range of 0-12 µg/m³, and the ozone level will be below the safe threshold of 0.075 ppm. This makes it a suitable day for outdoor activities, such as going for a walk or bike ride.


In [18]:
QUESTION4 = "What will the air quality be like the day after tomorrow?"

response4 = generate_response(
    QUESTION4,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response4)

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.64s) 

Based on the air quality measurements provided, the air quality level for the date 2024-11-17 is 48.93. This falls within the moderate range, which is generally considered safe for most people. It would be suitable for outdoor activities such as walking or jogging.


In [22]:
QUESTION5 = "What will the air quality be like this Sunday?"

response5 = generate_response(
    QUESTION5,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response5)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.82s) 

Based on the air quality measurements for your city, the air quality on this Sunday, November 17, is expected to be poor. The levels of PM2.5 and NO2 are higher than the recommended safe limits, which means it might not be the best day for outdoor activities like walking or running. It is advisable to limit your exposure to the outdoors and take necessary precautions if you have respiratory issues.


In [ ]:
QUESTION7 = "What will the air quality be like for the rest of the week?"

response7 = generate_response(
    QUESTION7,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response7)

In [ ]:
QUESTION = "Will the air quality be safe or not for the next week?"

response = generate_response(
    QUESTION7,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Is tomorrow's air quality level dangerous?"

response = generate_response(
    QUESTION,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [ ]:
QUESTION = "Can you please explain different PM2_5 air quality levels?"

response = generate_response(
    QUESTION,
    feature_view,
    weather_fg,
    model_air_quality,
    model_llm,
    tokenizer,
    llm_chain,
    verbose=False,
)

print(response)

In [23]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [24]:
!pip install openai --quiet
!pip install gradio==3.40.1 --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 118.2 MB/s eta 0:00:00


In [25]:
import gradio as gr
from transformers import pipeline
import numpy as np
from xgboost import XGBRegressor
from functions.llm_chain import load_model, get_llm_chain, generate_response


In [26]:
# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    if y.ndim > 1 and y.shape[1] > 1:
        y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]

def generate_query_response(user_query, method, openai_api_key=None):
    if method == 'Hermes LLM':
        response = generate_response(
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            model_llm,
            tokenizer,
            llm_chain,
            verbose=False,
        )
        return response

    elif method == 'OpenAI API' and openai_api_key:
        client = OpenAI(
            api_key=openai_api_key
        )

        response = generate_response_openai(
            user_query,
            feature_view,
            weather_fg,
            model_air_quality,
            client=client,
            verbose=True,
        )
        return response

    else:
        return "Invalid method or missing API key."

def handle_input(text_input=None, audio_input=None, method='Hermes LLM', openai_api_key=""):
    if audio_input is not None:
        user_query = transcribe(audio_input)
    else:
        user_query = text_input

    # Check if OpenAI API key is required but not provided
    if method == 'OpenAI API' and not openai_api_key.strip():
        return "OpenAI API key is required for this method."

    if user_query:
        return generate_query_response(user_query, method, openai_api_key)
    else:
        return "Please provide input either via text or voice."


# Setting up the Gradio Interface
iface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(placeholder="Type here or use voice input..."),
        gr.Audio(),
        gr.Radio(["Hermes LLM", "OpenAI API"], label="Choose the response generation method"),
        gr.Textbox(label="Enter your OpenAI API key (only if you selected OpenAI API):", type="password")  # Removed `optional=True`
    ],
    outputs="text",
    title="🌤️ AirQuality AI Assistant 💬",
    description="Ask your questions about air quality or use your voice to interact. Select the response generation method and provide an OpenAI API key if necessary."
)

iface.launch(share=True)


config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

IMPORTANT: You are using gradio version 3.40.1, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e8bce76f52f82e1b06.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


---